In [ ]:
# Necessary Dependencies
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import cv2
import datetime

import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.layers import Dense,Conv2D, Flatten, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import CSVLogger

%load_ext tensorboard

#from New_configs import *

print('Started')

# DenseNet

This model is based of Stanford Pneumonia Classification algorithm. In their paper they used a 121-layer convolutional neural network called CheXNet. Their network was able to classify 14 thoraric diseases as well as provide a heat map on images to help aid doctors in their diagnosis. 
- 70% TRAIN
- 10% VALIDATION
- 20% TEST

- Elementwise Sigmoid Nonlinearity



In [2]:
# HyperParameters
BATCH_SIZE = 6
EPOCHS = 10
STEPS_EPOCH = 6
LR = 0.001
METRICS = ['accuracy','binary_crossentropy', tf.keras.metrics.FalseNegatives(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
SEED = 1 
IMG_SIZE = 160
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [3]:
data_dir_train = '/Users/ryanjoseph/Desktop/Yale/Repositories/Pneumonia/Multi_Chest/Data/train/'
data_dir_val = '/Users/ryanjoseph/Desktop/Yale/Repositories/Pneumonia/Multi_Chest/Data/val/'

CLASS_NAMES = ['mass','cardiomegaly', 'atelectasis', 'effusion', 'pneumothorax', 'no_finding', 'nodule', 'infiltration', 'consolidation']
# 9 Classes

## Image Generators 
This are functions that do all the image pre-processing and augmentation. From there we send it straight to the Neural Network 

In [4]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = image_generator.flow_from_directory(directory=str(data_dir_train),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     seed=SEED,
                                                     target_size=(160, 160),
                                                     classes = list(CLASS_NAMES))

val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=str(data_dir_val),
                                                              seed=SEED,
                                                              target_size=(160, 160),
                                                              classes = list(CLASS_NAMES))

Found 23777 images belonging to 9 classes.
Found 5948 images belonging to 9 classes.


In [11]:
# CSV Logger
csv_logger = CSVLogger('training.log.csv')


# Tensorboard
tensorboard_logs="logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_logs, histogram_freq=1)

## DenseNet Architecture

In [ ]:
base_model = tf.keras.applications.DenseNet121(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
#base_model.summary()

In [ ]:
base_model.trainable = True

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(9, activation='sigmoid',name='Final')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

In [ ]:
#print(model.summary())

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=STEPS_EPOCH,
    epochs=EPOCHS,
    validation_data = val_data_gen,
    callbacks=[tensorboard_callback, csv_logger]
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#%tensorboard --logdir logs/

## Vanilla CNN

In [6]:
# Learning Model 
model_2 = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(160, 160 ,3)),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(9, activation='sigmoid')
])

In [7]:
model_2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 160, 160, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 80, 80, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 102400)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               52429312  
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 4617      
Total params: 52,434,377
Trainable params: 52,434,377
Non-trainable params: 0
_________________________________________________________________


In [8]:
model_2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model_2.fit_generator(
    train_data_gen,
    steps_per_epoch=STEPS_EPOCH,
    epochs=EPOCHS,
    validation_data = val_data_gen,
    callbacks=[tensorboard_callback, csv_logger]
)

Epoch 1/10
6/6 [==============================] - 52s 9s/step - loss: 2.0660 - accuracy: 0.8302 - val_loss: 2.5391 - val_accuracy: 0.8349
Epoch 2/10
5/6 [========================>.....] - ETA: 0s - loss: 2.8480 - accuracy: 0.8148

In [ ]:
%tensorboard --logdir logs/

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()